In [ ]:
## ADR

## Store all ADR + RSI above 50

In [38]:
import yfinance as yf
import pandas as pd
import os

# Function to calculate Average Daily Range (ADR)
def calculate_ADR(data):
    data['DailyHigh'] = data['High']
    data['DailyLow'] = data['Low']
    ADR_highlow = (data['DailyHigh'] / data['DailyLow']).rolling(window=14).mean()
    ADR_perc = 100 * (ADR_highlow - 1)
    return ADR_perc

# Function to calculate Modified_ADR as absolute percentage change
def calculate_modified_ADR(data):
    data['dr_pct'] = data.apply(lambda x: 100 * (x["High"] / x["Low"] - 1), axis=1)
    data["mod_adr"] = data['dr_pct'].rolling(window=20).mean()
    return data["mod_adr"]

# Function to calculate Relative Strength Index (RSI)
def calculate_RSI(data, window=14):
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Read the list of stocks from CSV file
stocks_df = pd.read_csv("../data/niftymidsmallcap400list_performers.csv")

# Get the directory and file name from the input file path
file_path = "../data/niftymidsmallcap400list_performers.csv"
file_name, file_extension = os.path.splitext(os.path.basename(file_path))

# Empty DataFrame to store results
plain_adr_results_list = []

# Loop through each stock and apply the conditions
for ticker in stocks_df['Symbol']:
    try:
        data = yf.download(ticker, start='2023-01-01', end='2024-03-17')
        adr = calculate_ADR(data)
        modified_adr = calculate_modified_ADR(data)
        rsi = calculate_RSI(data)

        # Check conditions: ADR > 5 and RSI > 30
        if adr.iloc[-1] > 5 and rsi.iloc[-1] > 30:
            plain_adr_results_list.append({'Stock': ticker,
                                           'Close': round(data['Close'].iloc[-1], 2),
                                           'Volume': round(data['Volume'].iloc[-1], 2),
                                           'ADR': round(adr.iloc[-1], 2),
                                           'Mod_ADR': round(modified_adr.iloc[-1], 2),
                                           'RSI': round(rsi.iloc[-1], 2)})
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Create DataFrame from results list
plain_adr_results_df = pd.DataFrame(plain_adr_results_list)

# Save the DataFrame to CSV file with appended text to the original file name
output_file_path = f"{file_name}_with_ADR_RSI.csv"
plain_adr_results_df.to_csv(output_file_path, index=False)

# Displaying the results
print(plain_adr_results_df)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

            Stock    Close     Volume   ADR  Mod_ADR    RSI
0   INDUSTOWER.NS   248.35   14024872  5.19     4.78  54.10
1          ITI.NS   259.95    5647031  6.80     6.06  31.71
2         GPPL.NS   205.85   12708758  7.14     6.91  46.77
3    REDINGTON.NS   211.65    5550846  5.33     5.32  51.18
4        SOBHA.NS  1243.45     426053  7.66     7.47  34.95
5   SWANENERGY.NS   569.30   12599099  9.39     7.99  32.83
6   PHOENIXLTD.NS  2646.85    1518565  6.37     5.78  40.20
7         NHPC.NS    82.95  160183221  5.81     5.60  37.24
8         SJVN.NS   124.60  119509864  7.64     6.78  54.13
9     NLCINDIA.NS   209.10   14114208  6.18     5.77  32.66
10         OIL.NS   555.65   17276038  5.96     6.39  41.80
11        MRPL.NS   215.65    8757494  7.63     8.23  35.93
12  CASTROLIND.NS   197.15    3891138  5.21     4.84  45.59
13  NATIONALUM.NS   136.40   41641387  5.40     5.21  31.95
14  HINDCOPPER.NS   271.15   39164127  5.74     5.37  53.15
15        SAIL.NS   121.70   38312218  5

## ADR Final

In [42]:
import yfinance as yf
import pandas as pd

# Function to calculate Average Daily Range (ADR)
def calculate_ADR(data):
    data['DailyHigh'] = data['High']
    data['DailyLow'] = data['Low']
    ADR_highlow = (data['DailyHigh'] / data['DailyLow']).rolling(window=14).mean()
    ADR_perc = 100 * (ADR_highlow - 1)
    return ADR_perc

# Function to calculate Modified_ADR as absolute percentage change
def calculate_modified_ADR(data):
    data['dr_pct'] = data.apply(lambda x: 100 * (x["High"] / x["Low"] - 1), axis=1)
    data["mod_adr"] = data['dr_pct'].rolling(window=20).mean()
    return data["mod_adr"]

# Function to apply the scanner conditions
def apply_scanner_conditions(stock_data):
    # Check if the Average Daily Range (ADR) is greater than 5%
    adr_above_5 = stock_data['ADR'] > 5
    # Check if the closing price is greater than 25% higher than the closing price 1 month ago
    price_greater_than_1M = stock_data['Close'] > stock_data['Close'].shift(22) * 1.25
    # Check if the closing price is greater than 50% higher than the closing price 3 months ago
    price_greater_than_3M = stock_data['Close'] > stock_data['Close'].shift(67) * 1.5
    # Check if the closing price is greater than 150% higher than the closing price 6 months ago
    price_greater_than_6M = stock_data['Close'] > stock_data['Close'].shift(126) * 2.5
    
    # Check if the closing price is within 85% of the 6-day high
    price_within_15_percent_of_high = stock_data['Close'] >= (stock_data['High'].rolling(window=6).max() * 0.85)

    # Check if the closing price is within 115% of the 6-day low
    price_within_15_percent_of_low = stock_data['Close'] <= (stock_data['Low'].rolling(window=6).min() * 1.15)

    # Calculate the Dollar Volume by multiplying the closing price with the volume
    stock_data['Dollar_Volume'] = stock_data['Close'] * stock_data['Volume']
    volume_greater_than_3M = stock_data['Dollar_Volume'] > 3000000
    scanner_results = adr_above_5 & price_greater_than_1M & price_greater_than_3M & price_greater_than_6M & \
                      price_within_15_percent_of_high & price_within_15_percent_of_low & volume_greater_than_3M
    return scanner_results

# Function to calculate Relative Strength Index (RSI)
def calculate_RSI(data, window=14):
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


# Read the list of stocks from CSV file
stocks_df = pd.read_csv("../data/nifty_smallcap_100_high_performers.csv")

# Empty DataFrame to store results
scanner_results_list = []

# Loop through each stock and apply the scanner conditions
for ticker in stocks_df['Symbol']:
    try:
        data = yf.download(ticker, start='2023-01-01', end='2024-03-17')
        data['ADR'] = calculate_ADR(data)
        data['Modified_ADR'] = calculate_modified_ADR(data)
        rsi = calculate_RSI(data)


        results = apply_scanner_conditions(data)
        if results.any():
            scanner_results_list.append({'Ticker': ticker,
                                         'Close': round(data['Close'].iloc[-1], 2),
                                         'ADR': round(data['ADR'].iloc[-1],2),
                                         'RSI': round(rsi.iloc[-1], 2),
                                         'Modified_ADR': round(data['Modified_ADR'].iloc[-1],2),
                                         'Volume': round(data['Volume'].iloc[-1],2)})
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Create DataFrame from results list
scanner_results = pd.DataFrame(scanner_results_list)

# Save the DataFrame to CSV file
scanner_results.to_csv('scanner_ADR_results.csv', index=False)

# Displaying the results
print(scanner_results)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

          Ticker    Close   ADR    RSI  Modified_ADR    Volume
0        MRPL.NS   215.65  7.63  35.93          8.23   8757494
1       HUDCO.NS   188.80  7.76  44.68          6.91  42797939
2  KALYANKJIL.NS   370.05  5.74  43.97          5.09   4403727
3        NBCC.NS   111.85  5.35  17.90          5.16  17995457
4         IRB.NS    57.30  6.12  32.38          5.35  36564735
5  COCHINSHIP.NS   890.45  5.99  54.29          5.27  11812467
6        JBMA.NS  1892.00  6.44  30.49          6.59    200381
